In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import urllib
from time import sleep

In [2]:
# define a function to get the soup
def get_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

# define a function to increment the url to the next page

def next_page(base, player_count):
    url = base+str(player_count)
    return url

# define a function that will return a list of the rows of player stats on a page
def get_rows(soup):
    body = soup.findAll('tbody')
    rows = body[0].findAll('tr')
    #stat_rows = [x for x in rows if x['td']==['']]
    return rows

# define a function that will strip the stats out of a row of player html data
def strip_stats(rows):
    for x in range(0,len(rows)):
        player_stats = rows[x].findAll('td')
        stats.append([z.text for z in player_stats])

# Standard Box Score Statistics  

Here I will go through and collect basic statistics for forwards

In [3]:
base = ('http://www.hockey-reference.com/play-index/psl_finder.cgi?request=1&match=single&year_min=&year_max=&birthyear_min=&birthyear_max=&threshhold=5&season_end=-1&age_min=0&age_max=99&birth_country=&franch_id=&is_active=&is_hof=&is_playoffs=N&pos=F&handed=&c1stat=games_played&c1comp=gt&c1val=&c2stat=goals&c2comp=gt&c2val=&c3stat=shots&c3comp=gt&c3val=&c4stat=ops&c4comp=gt&c4val=&order_by=time_on_ice&order_by_asc=&offset=')

# The master function will need to have a stats and counter that increments by 200
# and prints out how many players it has processed per loop
def get_forward_stats(base_url, limit):
    player_count = 0
    while player_count <= limit:
        url = next_page(base_url, player_count)
        soup = get_soup(url)
        rows = get_rows(soup)
        strip_stats(rows)
        player_count +=200
        print '%d players processed' % player_count
        #sleep(.5)

In [4]:
# instantiate the list that the player stats will be added to. I've yet to figure
# out how to have this list exist inside the function and be returned by it.
stats = []

In [5]:
# after looking through the results of the search on pro-football-reference, this
# upper limit will capture all of the players in the search results.
get_forward_stats(base, 7200)

200 players processed
400 players processed
600 players processed
800 players processed
1000 players processed
1200 players processed
1400 players processed
1600 players processed
1800 players processed
2000 players processed
2200 players processed
2400 players processed
2600 players processed
2800 players processed
3000 players processed
3200 players processed
3400 players processed
3600 players processed
3800 players processed
4000 players processed
4200 players processed
4400 players processed
4600 players processed
4800 players processed
5000 players processed
5200 players processed
5400 players processed
5600 players processed
5800 players processed
6000 players processed
6200 players processed
6400 players processed
6600 players processed
6800 players processed
7000 players processed
7200 players processed
7400 players processed


In [6]:
len(stats)

forward_cols = ['player', 'team', 'lg', 'pos', 'season', 'age',
                'games_played', 'goals', 'assists', 'points', 'plus_minus',
               'pen_mins', 'even_goals', 'pp_goals', 'sh_goals','gw_goals',
               'shots', 'shot_%', 'toi', 'g/game', 'a/game', 'pts/game',
               'shots/game', 'off_point_shares', 'def_point_shares',
               'tot_point_shares']

In [7]:
df = pd.DataFrame(stats, columns = forward_cols)
print df.shape
#print df.dtypes
df.tail()


(7231, 26)


,player,team,lg,pos,season,age,games_played,goals,assists,points,...,shots,shot_%,toi,g/game,a/game,pts/game,shots/game,off_point_shares,def_point_shares,tot_point_shares
7226,Andrei Vasilyev,PHX,NHL,LW,1998-99,26,1,0,0,0,...,0,,1,0.00,0.00,0.00,0.00,0.0,-0.1,-0.1
7227,Brian Wesenberg,PHI,NHL,RW,1998-99,21,1,0,0,0,...,0,,1,0.00,0.00,0.00,0.00,0.0,0.0,0.0
7228,B.J. Young,DET,NHL,RW,1999-00,22,1,0,0,0,...,1,0.0,1,0.00,0.00,0.00,1.00,0.0,0.0,0.0
7229,Ryan Flinn,LAK,NHL,LW,2005-06,25,2,0,0,0,...,0,,1,0.00,0.00,0.00,0.00,0.0,0.0,0.0
7230,Greg Koehler,CAR,NHL,C,2000-01,25,1,0,0,0,...,0,,1,0.00,0.00,0.00,0.00,0.0,0.0,0.0


In [8]:
df.season.value_counts()

2003-04    452
2015-16    442
2008-09    440
2001-02    437
2010-11    433
2002-03    427
2011-12    426
2013-14    426
2009-10    425
2005-06    425
2007-08    424
1999-00    424
2014-15    423
2012-13    419
2000-01    413
2006-07    402
1998-99    393
Name: season, dtype: int64

In [9]:
numeric_cols = ['age',
                'games_played', 'goals', 'assists', 'points', 'plus_minus',
               'pen_mins', 'even_goals', 'pp_goals', 'sh_goals','gw_goals',
               'shots', 'shot_%', 'toi', 'g/game', 'a/game', 'pts/game',
               'shots/game', 'off_point_shares', 'def_point_shares',
               'tot_point_shares']

for col in numeric_cols:
    df[col] = df[col].convert_objects(convert_numeric = True)
    
print df.dtypes

player               object
team                 object
lg                   object
pos                  object
season               object
age                   int64
games_played          int64
goals                 int64
assists               int64
points                int64
plus_minus            int64
pen_mins              int64
even_goals            int64
pp_goals              int64
sh_goals              int64
gw_goals              int64
shots                 int64
shot_%              float64
toi                   int64
g/game              float64
a/game              float64
pts/game            float64
shots/game          float64
off_point_shares    float64
def_point_shares    float64
tot_point_shares    float64
dtype: object


/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


# Advanced Stats for Forwards

In [14]:
# define a function to get the soup
def get_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

# define a function to increment the url to the next page

def next_page(base, player_count):
    url = base+str(player_count)
    return url

# define a function that will return a list of the rows of player stats on a page
def get_rows(soup):
    body = soup.findAll('tbody')
    rows = body[0].findAll('tr')
    #stat_rows = [x for x in rows if x['td']==['']]
    return rows

# define a function that will strip the stats out of a row of player html data
def adv_strip_stats(rows):
    for x in range(0,len(rows)):
        player_stats = rows[x].findAll('td')
        adv_stats.append([z.text for z in player_stats])

In [24]:
adv_base = ('http://www.hockey-reference.com/play-index/ppbp_finder.cgi?c1comp=gt&c1stat=corsi_for&c1val=&c2comp=gt&c2stat=fenwick_for&c2val=&c3comp=gt&c3stat=on_ice_shot_pct&c3val=&c4comp=gt&c4stat=zs_offense_pct&c4val=&age_min=0&age_max=99&birth_country=&handed=&pos=F&rookie=N&season_start=1&season_end=-1&situation_id=all&franch_id=&year_min=&year_max=&match=single&order_by=hits&order_by_asc=&request=1&offset=')
adv_stats = []
def get_adv_forward_stats(base_url, limit):
    player_count = 0
    while player_count <= limit:
        url = next_page(base_url, player_count)
        soup = get_soup(url)
        rows = get_rows(soup)
        adv_strip_stats(rows)
        player_count +=100
        print '%d players processed' % player_count

get_adv_forward_stats(adv_base, 5100)

100 players processed
200 players processed
300 players processed
400 players processed
500 players processed
600 players processed
700 players processed
800 players processed
900 players processed
1000 players processed
1100 players processed
1200 players processed
1300 players processed
1400 players processed
1500 players processed
1600 players processed
1700 players processed
1800 players processed
1900 players processed
2000 players processed
2100 players processed
2200 players processed
2300 players processed
2400 players processed
2500 players processed
2600 players processed
2700 players processed
2800 players processed
2900 players processed
3000 players processed
3100 players processed
3200 players processed
3300 players processed
3400 players processed
3500 players processed
3600 players processed
3700 players processed
3800 players processed
3900 players processed
4000 players processed
4100 players processed
4200 players processed
4300 players processed
4400 players process

In [25]:
adv_stats

[[u'Matt Martin',
  u'LW',
  u'NYI',
  u'2014-15',
  u'78',
  u'765',
  u'715',
  u'51.7',
  u'-2.0',
  u'3.4',
  u'-5.3',
  u'578',
  u'501',
  u'53.6',
  u'-1.3',
  u'6.4',
  u'91.5',
  u'97.9',
  u'46.7',
  u'53.3',
  u'11.3',
  u'5',
  u'5',
  u'50.0',
  u'382',
  u'36',
  u'24',
  u'27'],
 [u'Matt Martin',
  u'LW',
  u'NYI',
  u'2011-12',
  u'80',
  u'751',
  u'979',
  u'43.4',
  u'-7.4',
  u'-14.1',
  u'-14.9',
  u'573',
  u'689',
  u'45.4',
  u'-6.0',
  u'3.9',
  u'91.1',
  u'95.1',
  u'41.5',
  u'58.5',
  u'12.2',
  u'10',
  u'13',
  u'43.5',
  u'374',
  u'41',
  u'36',
  u'21'],
 [u'Matt Martin',
  u'LW',
  u'NYI',
  u'2015-16',
  u'80',
  u'716',
  u'731',
  u'49.5',
  u'-1.9',
  u'-1.1',
  u'-2.6',
  u'517',
  u'533',
  u'49.2',
  u'-2.9',
  u'10.1',
  u'91.4',
  u'101.5',
  u'41.6',
  u'58.4',
  u'10.6',
  u'1',
  u'3',
  u'25.0',
  u'365',
  u'24',
  u'8',
  u'15'],
 [u'Matt Martin',
  u'LW',
  u'NYI',
  u'2013-14',
  u'79',
  u'739',
  u'964',
  u'43.4',
  u'-10.2',
  u'-

In [48]:
adv_forw_cols = ['player', 'pos', 'team', 'season', 'games_played', 'corsi_for',
                'corsi_ag', 'corsi_for%', 'corsi_for%_rel', 'corsi/60',
                'corsi_rel/60', 'fen_for', 'fen_ag', 'fen_for%', 'fen_for%_rel',
                'team_onice_sh%', 'team_onice_sv%', 'pdo', 'off_zs%', 'def_zs%',
                'toi/game', 'faceoffs_won', 'faceoffs_lost', 'faceoff%',
                'hits', 'blocks', 'takeaways', 'giveaways']

adv_df = pd.DataFrame(adv_stats, columns = adv_forw_cols)
adv_df.tail()

,player,pos,team,season,games_played,corsi_for,corsi_ag,corsi_for%,corsi_for%_rel,corsi/60,...,off_zs%,def_zs%,toi/game,faceoffs_won,faceoffs_lost,faceoff%,hits,blocks,takeaways,giveaways
5373,Brandon Saad,LW,CHI,2011-12,2,17,26,39.5,-21.3,-19.3,...,70.0,30.0,14.0,0,0,,0,1,2,0
5374,Zach Hamill,C,BOS,2010-11,3,15,21,41.7,-10.9,-11.5,...,46.2,53.8,10.5,5,11,31.3,0,3,1,2
5375,Cory Emmerton,C,DET,2010-11,2,12,7,63.2,8.3,18.0,...,55.6,44.4,8.3,0,1,0.0,0,0,0,0
5376,Adam Henrique,C,NJD,2010-11,1,11,8,57.9,4.0,13.5,...,55.6,44.4,13.4,0,1,0.0,0,2,0,0
5377,Tyler Graovac,C,MIN,2014-15,3,17,24,41.5,-9.6,-15.2,...,50.0,50.0,9.2,6,16,27.3,0,2,2,0


In [27]:
adv_numeric_cols = ['games_played', 'corsi_for',
                'corsi_ag', 'corsi_for%', 'corsi_for%_rel', 'corsi/60',
                'corsi_rel/60', 'fen_for', 'fen_ag', 'fen_for%', 'fen_for%_rel',
                'team_onice_sh%', 'team_onice_sv%', 'pdo', 'off_zs%', 'def_zs%',
                'toi/game', 'faceoffs_won', 'faceoffs_lost', 'faceoff%',
                'hits', 'blocks', 'takeaways', 'giveaways']

for i in adv_numeric_cols:
    adv_df[i] = adv_df[i].convert_objects(convert_numeric = True)
    
adv_df.dtypes

/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


player             object
pos                object
team               object
season             object
games_played      float64
corsi_for         float64
corsi_ag          float64
corsi_for%        float64
corsi_for%_rel    float64
corsi/60          float64
corsi_rel/60      float64
fen_for           float64
fen_ag            float64
fen_for%          float64
fen_for%_rel      float64
team_onice_sh%    float64
team_onice_sv%    float64
pdo               float64
off_zs%           float64
def_zs%           float64
toi/game          float64
faceoffs_won      float64
faceoffs_lost     float64
faceoff%          float64
hits              float64
blocks            float64
takeaways         float64
giveaways         float64
dtype: object